In [1]:
import pandas as pd

In [4]:
data = pd.read_csv('data_berita.csv')
data.head()

,entryId,title,subtitle,content,displayDate,tag
0,288434,4 Alasan Kenapa Opsi SNMPTN Layak Dilirik,Tahukah Kamu?,<p>Kamu merasa tak punya peluang untuk ikut SN...,2018-07-30 11:48:00,snmptn
1,492858,Skema Pembiayaan Perkuliahan pada 8 PTN,Info Terbaru,<p>Tak cuma pilihan jurusan yang musti dipersi...,2018-08-24 07:26:00,snmptn
2,1141875,"Yuk, Ketahui Seluk Beluk SNMPTN",SNMPTN Tes Masuk PTN Jalur Undangan,<p>Perubahan SBMPTN menjadi <i>Test Center</i>...,2018-09-30 07:56:00,snmptn
3,7312259,Kenali Lembaga Ini Sebelum Ujian: LTMPT,Ujian Masuk PTN,"<p>Mulai tahun 2019, Kemenristekdikti akan men...",2018-11-10 03:08:00,snmptn
4,14056533,Tiga Pintu Masuk ITB: Kini dengan Seleksi Mandiri,Ujian Masuk PTN,<p>\n <i>Lucky you</i>! Selama tiga tahun bel...,2018-12-15 14:06:00,snmptn


In [9]:
#Data Shape
len(data.entryId.unique())

721

In [6]:
#Count Tag
data.tag.value_counts()

kuliah               325
aku pintar           317
mahasiswa            314
sobat pintar         310
pelajar              176
                    ... 
kip                    1
tps                    1
unsyiah                1
fakultas mipa          1
olahraga rekreasi      1
Name: tag, Length: 673, dtype: int64

In [7]:
#Unique Tag
len(data.tag.unique())

673

In [18]:
def normalize(text):
    text = re.sub('<[^<]+?>', '', text) #remove tag
    text = text.lower() #lower
    text = re.sub(r'[^\w]',' ',text) #remove punct
    text = re.sub(r'\s+', ' ', text) #remove white space
#     text = re.sub(r'[,.;`:]', '', text)
    
    return text
    
#     re.sub('<[^<]+?>', '', text)

In [19]:
normalize(data.content.values[0])

'kamu merasa tak punya peluang untuk ikut snmptn kenapa nilai rapormu naik turun seperti ombak di pantai tunggu dulu jangan lekas patah hati baca dulu barangkali kamu masih punya peluang 1 kriteria seleksi snmptn bahasa kerennya seleksi snmptn dilakukan melalui penelusuran prestasi akademik intinya nilai rapormu dari semester satu sampai semester lima yang akan jadi acuan utama untuk menentukan apakah kamu akan diterima di ptn atau tidak secara umum nilai akademik yang konsisten tinggi punya peluang besar untuk lolos seleksi tapi jika nilai rapormu sempat jatuh misalnya karena pada suatu semester kamu sakit dan dirawat cukup lama di rumah sakit coba saja snmptn mu berharaplah portofolio akademik yang mengiringi rapormu bisa membantu 2 prestasi non akademik selain pretasi akademik seleksi snmptn juga akan menimbang prestasimu dibidang non akademik pernah ikut dan jadi juara debat juara lomba tari tradisional atau modern scan dan lantas lampirkan saja semua sertifikat atau piagam non akd